# Building a RAG-Integrated Chatbot (LangGraph Version)

## 1. Setup

In [1]:
import uuid
from pathlib import Path
from typing import Any, Annotated, TypedDict

from dotenv import load_dotenv

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_core.messages import AIMessageChunk, BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

load_dotenv()

MODEL_NAME = "gpt-5-mini"

## 2. Prepare Text Data

In [2]:
TXT_DIR = Path("data/txt")
TXT_DIR.mkdir(parents=True, exist_ok=True)

# Keep temperature low to reduce output variance
llm = ChatOpenAI(model=MODEL_NAME, temperature=0.2)
parser = StrOutputParser()

policy_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert in drafting internal corporate policies and employee guidelines."
     "You will create internal policy documents for a fictional company in English."
     "Do NOT include real company names, real people, or personal data—everything must be entirely fictional."
     "Output should be clear and easy to read, using headings, bullet points, and numbered sections (e.g., 'Section 1', '1.1', etc.)."
     "Avoid vague language. Clearly specify exceptions, prohibited actions, approval workflows, required documentation, deadlines, and enforcement."
     "Include a revision history at the end, for example: 'Revision History: v1.0 — 2025-01-01'."
     "Do not add unnecessary preface text; output only the policy body."
    ),
    ("human",
     "Company Name: {company_name}\n"
     "Industry: {industry}\n"
     "Company Size: {size}\n"
     "Document Title: {title}\n"
     "Topics to Include: {topics}\n"
     "Tone: {tone}\n"
     "Length: {length}\n"
    ),
])

chain = policy_prompt | llm | parser

company_profile = {
    "company_name": "Aurora Works, Inc.",
    "industry": "B2B SaaS (business productivity software)",
    "size": "150 employees (primarily U.S.-based, hybrid/remote-friendly)",
    "tone": "Clear and practical. Do not be vague about exceptions or prohibited conduct.",
    "length": "About 2–3 pages (detailed enough for real operations, but not overly long).",
}

docs_to_generate = [
    {
        "filename": "employee_handbook.txt",
        "title": "Employee Handbook (Excerpt)",
        "topics": (
            "Work hours, flexible scheduling, core hours (if applicable), remote work, leave and time off, "
            "tardiness and early departures, outside employment/moonlighting, performance reviews, "
            "disciplinary actions, and reporting/HR contact channels"
        ),
    },
    {
        "filename": "expense_policy.txt",
        "title": "Expense Reimbursement Policy",
        "topics": (
            "Local transportation, business travel, lodging, per diem (if applicable), client meals/entertainment, "
            "receipts, submission deadlines, exception approvals, and consequences for violations"
        ),
    },
    {
        "filename": "security_policy.txt",
        "title": "Information Security Policy",
        "topics": (
            "Data classification, passwords and MFA, device management, removal of company assets/data, "
            "approved cloud services, incident reporting, and prohibited activities"
        ),
    },
]

for spec in docs_to_generate:
    text = chain.invoke({
        **company_profile,
        "title": spec["title"],
        "topics": spec["topics"],
    })
    (TXT_DIR / spec["filename"]).write_text(text, encoding="utf-8")
    print("Saved:", TXT_DIR / spec["filename"])


Saved: data\txt\employee_handbook.txt
Saved: data\txt\expense_policy.txt
Saved: data\txt\security_policy.txt


## 3. Create the Index

In [3]:
# 1. Load (supports multiple files)
loader = DirectoryLoader(
    path="data/txt",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True,
)
documents = loader.load()

# 2. Split (chunking)
# Adding Japanese-friendly separators tends to stabilize splitting.
splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=150,
    separators=["\n\n", "\n", "。", "！", "？", "、", " ", ""],
)
chunks = splitter.split_documents(documents)

# 3. Embedding, 4. Indexing
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(chunks)

retriever = vector_store.as_retriever(search_kwargs={"k": 4})

print(f"Loaded docs: {len(documents)} / chunks: {len(chunks)}")

100%|██████████| 3/3 [00:00<00:00, 27.66it/s]


Loaded docs: 3 / chunks: 55


## 4. Create the RAG Retrieval Tool

In [4]:
def format_docs(docs) -> str:
    lines = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("source", "unknown")
        src_name = Path(src).name if isinstance(src, str) else "unknown"
        lines.append(f"[{i}] source: {src_name}\n{d.page_content}")
    return "\n\n".join(lines) if lines else "(No relevant results)"

@tool
def rag_search(query: str) -> str:
    """Search the (fictional) Aurora Works internal policy TXT files and return relevant excerpts."""
    docs = retriever.invoke(query)
    return format_docs(docs)

tools = [rag_search]


## 5. Turn It Into a RAG Chatbot with LangGraph

In [5]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# Common prompt (explicitly scoped to Aurora Works)
prompt_text = """
You are an assistant that answers based on the internal policies of the fictional company "Aurora Works."

Rules:
- For any question about Aurora Works (employment, expenses, security, or other company policies), you MUST use rag_search to verify the source before answering.
- For general knowledge questions that are NOT about the company, answer normally without using any tools.
- If you use rag_search:
  - Append citation numbers at the end of your answer (e.g., [1][2]).
  - Then add a "References:" section and list each number with its corresponding file name as bullet points.
- If you cannot find supporting evidence in the retrieved materials, do not guess—respond with: "I couldn't find supporting evidence in the provided materials."
- Do not paste large blocks of retrieved text; summarize only the key points.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_text),
    MessagesPlaceholder(variable_name="messages"),
])

# Prepare two LLM paths
# 1) For normal answers (no tools)
model_plain = ChatOpenAI(model=MODEL_NAME, temperature=0.2)
chain_plain = prompt | model_plain

# 2) For company questions → force tool calling (tools enabled & required)
# Since there is only one tool (rag_search), required is easy to work with.
model_force_tool = ChatOpenAI(model=MODEL_NAME, temperature=0.2).bind_tools(
    tools,
    tool_choice="required",
)
chain_force_tool = prompt | model_force_tool

def is_company_question(text: str) -> bool:
    """
    Lightweight check for whether this question is about the company's policies (Aurora Works).
    *Simple implementation for learning. In production, tune keywords based on logs.
    """
    t = text.lower()

    keywords = [
        "aurora works", "internal policy", "company policy", "employee handbook", "handbook",
        "employment", "work hours", "working hours", "attendance", "clock in", "clock out",
        "leave", "pto", "paid time off", "vacation", "sick leave", "absence", "tardy", "late", "early departure",
        "flex", "flex time", "flexible schedule", "core hours", "remote", "work from home", "wfh", "hybrid",
        "expense", "reimbursement", "expense report", "receipt", "travel", "business trip", "lodging", "per diem", "client meal",
        "security", "confidential", "confidentiality", "password", "mfa", "device", "laptop", "asset", "data removal",
        "incident", "breach", "discipline", "performance review", "moonlighting", "outside employment", "hr", "reporting channel",
    ]
    return any(k in t for k in keywords)

def chatbot(state: State) -> dict[str, Any]:
    """
    Key adjustments:
    - If the latest message is a user message, decide "company question or not?"
      - Company question: chain_force_tool (force tool call)
      - Otherwise: chain_plain (answer normally)
    - If the latest message is a tool result (ToolMessage), use chain_plain for the final answer
      (prevents a tool re-call loop)
    """
    messages = state["messages"]
    last = messages[-1]

    last_type = type(last).__name__  # ToolMessage / HumanMessage / AIMessage, etc.

    if last_type == "HumanMessage":
        user_text = getattr(last, "content", "") or ""
        if is_company_question(user_text):
            response = chain_force_tool.invoke(state)
        else:
            response = chain_plain.invoke(state)
    else:
        # If ToolMessage is present, etc., treat this as final-answer phase and summarize with plain.
        response = chain_plain.invoke(state)

    return {"messages": [response]}

tool_node = ToolNode(tools)

# Build the graph (do not change the structure)
builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_node("tools", tool_node)

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")

memory = InMemorySaver()
rag_agent = builder.compile(checkpointer=memory)

print("Ready.")


Ready.


## 6. Main Loop

In [6]:
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    user_input = input("Enter a message: ")
    if user_input.strip() == "":
        break

    print(f"Question: {user_input}")

    is_spinning = False

    for chunk, metadata in rag_agent.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config,
        stream_mode="messages",
    ):
        if isinstance(chunk, AIMessageChunk):
            if chunk.tool_call_chunks:
                if chunk.tool_call_chunks[-1]["name"] is not None:
                    print(f"[DEBUG] Tool call: {chunk.tool_call_chunks[-1]['name']}", flush=True)
                    is_spinning = True
                continue

            if chunk.content:
                if is_spinning:
                    print()
                    is_spinning = False
                print(chunk.content, end="", flush=True)

    print()

print("\n--- Thank you for using this! ---")


Question: Do you offer a flex-time policy?
[DEBUG] Tool call: rag_search

Yes — Aurora Works supports a flex-time (flexible scheduling) policy.

Key points:
- Flexible start/stop times are allowed outside core hours to accommodate personal needs and different time zones.
- How to request: submit a Flexible Schedule Request in the HR portal at least 10 business days before the requested start.
- Approval workflow and timelines:
  - Manager reviews and approves, proposes modifications, or denies within 5 business days.
  - HR reviews for policy compliance and finalizes within 3 business days.
- Exceptions: roles that require fixed coverage (for example scheduled customer support) may be ineligible; a manager must document the rationale when denying a request.
- Applicability and hours tracking:
  - Policy applies to all employees in the U.S. where local law allows.
  - Exempt employees are expected to meet a 40-hour work week; non‑exempt employees must record scheduled hours and obtain p